In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import glob
import csv
from datetime import datetime


# Creating points on a sphere 

In [2]:
# Inputs
theta_max = 40      # angle above horizontal (0-90 deg)
theta_min = 70      # angle below horizontal (0-90 deg, can be positive or negative)
n = 10000             # n = number of points on the sphere (full sphere, before cutting off range of vision)
 
golden_angle = np.pi * (3 - np.sqrt(5))
theta = golden_angle * np.arange(n)      # create an array with n elements
z = np.linspace(1 - 1.0 / n, 1.0 / n - 1, n)     # array with linear spacing from 1-1/n to 1/n-1 with n elements 
radius = np.sqrt(1 - z * z)
 
#selectTheta = np.array[(theta > theta.min) & (theta < theta.max)]

# Create points = array of n elements with [0, 0, 0]
points = np.zeros((n, 3))

points[:,0] = radius * np.cos(theta)
points[:,1] = radius * np.sin(theta)
points[:,2] = z

#select only points within range of vision
angle_max = abs(theta_max)/90
angle_min = -abs(theta_min)/90
selectPts = np.delete(points, np.where(np.logical_or(points[:,2]>angle_max, points[:,2]<=angle_min)), axis=0)

print(selectPts.shape)

# Convert the rays array to dataframe
all_rays_df = pd.DataFrame(selectPts)


(6111, 3)


In [3]:
# Plot test
pts_x = selectPts[:,0]
pts_y = selectPts[:,1]
pts_z = selectPts[:,2]
 
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter3D(pts_x, pts_y, pts_z)


# Functions for prepping BIN Directories

In [4]:
# FUNCTION: Concatenate all the pts for all floors into one dataframe
def concatenateAllPts(sourceDir, BIN):
   
    #sourceDir: the directory for all BIN folders
    #BIN: name of BIN (string)

    #Make list of all .pts file in directory
    os.chdir(sourceDir + "\\" + BIN)
    all_lines = []
    for file in glob.glob("*.pts"): 

        # Read .pts file into list and convert to DF
        with open(sourceDir + "\\" + BIN + "\\" + file) as f: 
            lines = f.readlines()
        lines = [x.strip() for x in lines]
        for i in range(len(lines)): 
            oneRow = list(map(float, lines[i].split()))
            del oneRow[-3:]
            all_lines.append(oneRow)

    #Create dataframe with points for all floors in BIN
    BIN_pts_df = pd.DataFrame(all_lines)
    BIN_pts_df.columns = ["x", "y", "z"]

    #print(BIN_pts_df.head())
    print("---> # of points in all floors in BIN (# rows):", BIN_pts_df.shape)
    #print(BIN_pts_df.shape)

    #Check that all floors are represented
    #print("z value of all floors:")
    #print(BIN_pts_df["z"].unique())
    
    return(BIN_pts_df)

# FUNCTION: Create the rays for each point in the BIN grids
def createGridRays(ray_vec_df, BIN_pts_df):

    # ray_vec_df: the trace to trace in a dataframe 
    # BIN_pts_df: all points in the BIN in a dataframe
    
    # Duplicate each row of grid point df by size of vector df and add ray index column
    vectors_df = pd.DataFrame(np.repeat(BIN_pts_df.values, len(ray_vec_df.index), axis=0))
    vectors_df['ray_count'] = vectors_df.groupby(0).cumcount()
    vectors_df = vectors_df.reset_index(drop=True)

    all_vectors = pd.merge(vectors_df, ray_vec_df, left_on='ray_count', right_index=True)
    all_vectors = all_vectors.drop(['ray_count'], axis=1)
    grid_vectors = all_vectors.sort_index()

    print("---> shape of grid df (# points x # rays):", grid_vectors.shape)
    #print(grid_vectors.shape)
    
    return(grid_vectors)

# FUNCTION: Export dataframe to CSV (TAKES A LONG TIME -- V BIG FILES!)
def writeGridRaystoCSV(grid_vectors, sourceDir, BIN):

    #grid_vectors: the output from createGridRays
    #sourceDir: the directory for all BIN folders
    #BIN: name of BIN (string)
    
    print("---> WRITE_START:", datetime.now())
    csvFile = sourceDir + "\\" + BIN + "\\" + BIN + "_rays.csv"
    grid_vectors.to_csv(csvFile, sep=" ", header=False, index=False)
    print("---> WRITE_END:", datetime.now())

    # Check the file size
    size = os.path.getsize(csvFile)
    print("---> csv file size: " + str(size/1e+6) + "mb")

# FUNCTION: Plot test the grid vectors 
def plotGridVectorsTest(grid_vectors):
    
    #Take a small portion of the vectors 
    grid_slice = grid_vectors.truncate(after=600)

    fig=plt.figure(figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')
    ax = fig.add_subplot(111, projection='3d')
    ax.quiver(grid_slice["0_x"], grid_slice["1_x"], grid_slice["2_x"], grid_slice["0_y"], grid_slice["1_y"], grid_slice["2_y"])

    #ax.set_zlim(-1.01, 1.01)

    ax.view_init(azim=0, elev=90)
    plt.show()


# Create csv with ray for full floor analysis grid --- all floors in BIN

In [5]:
#NYCsourceDir = r"C:\Users\ituran\NYCstudy\BINS"
NYCsourceDir = r"D:\ituran\NYCviews\BINS_to_do"

# Loop through all BINS in directory 
for file in os.listdir(NYCsourceDir): 
    print("---")
    print(file+"_START", datetime.now())
    
    print(file + "_1 concatenate all points in BIN")
    # made dataframe with the points on all floors in the BIN
    allflr_pt_df = concatenateAllPts(NYCsourceDir, file)

    print(file + "_2 make vector for all points in BIN")
    # apply vector rays to all points on all floors in the BIN
    allflr_vector_df = createGridRays(all_rays_df, allflr_pt_df)
    
    print(file + "_3 write vector df to CSV")
    # write the vector array to a file
    writeGridRaystoCSV(allflr_vector_df, NYCsourceDir, file)

    print(file+"_END", datetime.now())
    print("---")

---
1034199_START 2019-12-11 18:10:46.306846
1034199_1 concatenate all points in BIN
---> # of points in all floors in BIN (# rows): (8115, 3)
1034199_2 make vector for all points in BIN
---> shape of grid df (# points x # rays): (23887899, 6)
1034199_3 write vector df to CSV
---> WRITE_START: 2019-12-11 18:11:10.734732
---> WRITE_END: 2019-12-11 18:16:14.108185
---> csv file size: 2543.577594mb
1034199_END 2019-12-11 18:16:14.108185
---
---
1034200_START 2019-12-11 18:16:14.108185
1034200_1 concatenate all points in BIN
---> # of points in all floors in BIN (# rows): (1071, 3)
1034200_2 make vector for all points in BIN
---> shape of grid df (# points x # rays): (2236626, 6)
1034200_3 write vector df to CSV
---> WRITE_START: 2019-12-11 18:16:16.718489
---> WRITE_END: 2019-12-11 18:16:45.621353
---> csv file size: 239.587671mb
1034200_END 2019-12-11 18:16:45.621353
---
---
1034205_START 2019-12-11 18:16:45.621353
1034205_1 concatenate all points in BIN
---> # of points in all floors in

# Create batch file in each BIN folder

In [ ]:
BINSourceDir = r'D:\ituran\NYCviews\testFolder'
source_batch = r"D:\ituran\NYCviews\SimFiles\views_batch_master_D.bat"

# New input text
input_scene_cd = r'D:\ituran\NYCviews\RadScene'
input_oct = r'D:\ituran\NYCviews\RadScene\NYCmodel_all_wSky_r1.oct'

for file in os.listdir(BINSourceDir): 
    
    # Read the source batch file template
    with open(source_batch, "r") as f: 
        sourceFile = f.readlines()

    for i in range(len(sourceFile)): 
        sourceFile[i] = sourceFile[i].replace('cd', 'cd ' + input_scene_cd)
        sourceFile[i] = sourceFile[i].replace('set octreefile=', 'set octreefile=' + input_oct)
        sourceFile[i] = sourceFile[i].replace('set rayFile=', 'set rayFile=' + BINSourceDir + '\\' + file + '\\' + file + '_rays.csv')
        sourceFile[i] = sourceFile[i].replace('set resultsFile=', 'set resultsFile=' + BINSourceDir + '\\' + file + '\\' + file + '_out.dat')

    newBat = BINSourceDir + '\\' + file + '\\' 'view_batch_d.bat'
    with open(newBat, 'w') as out: 
        for i in sourceFile: 
            out.write(i)

                                          

# OLD CODE BELOW!

In [ ]:
# Loop through all .pts files in the directory (glob does not keep order!)
BINdir = r"C:\Users\ituran\NYCstudy\testFolder" 
BIN = str(1000007)

print(str(BINdir + "\\" + BIN))

In [ ]:
#Make list of all .pts file in directory
os.chdir(BINdir + "\\" + BIN)
all_lines = []
for file in glob.glob("*.pts"): 
    
    # Read .pts file into list and convert to DF
    with open(BINdir + "\\" + BIN + "\\" + file) as f: 
        lines = f.readlines()
    lines = [x.strip() for x in lines]
    for i in range(len(lines)): 
        oneRow = list(map(float, lines[i].split()))
        del oneRow[-3:]
        all_lines.append(oneRow)

#Create dataframe with points for all floors in BIN
BIN_pts_df = pd.DataFrame(all_lines)
BIN_pts_df.columns = ["x", "y", "z"]

#print(BIN_pts_df.head())
print(BIN_pts_df.shape)

#Check that all floors are represented
print(BIN_pts_df["z"].unique())

In [ ]:
# Convert the rays array to dataframe
ray_vec_df = pd.DataFrame(selectPts)

# Duplicate each row of grid point df by size of vector df and add ray index column
vectors_df = pd.DataFrame(np.repeat(BIN_pts_df.values, len(ray_vec_df.index), axis=0))
vectors_df['ray_count'] = vectors_df.groupby(0).cumcount()
vectors_df = vectors_df.reset_index(drop=True)

all_vectors = pd.merge(vectors_df, ray_vec_df, left_on='ray_count', right_index=True)
all_vectors = all_vectors.drop(['ray_count'], axis=1)
grid_vectors = all_vectors.sort_index()

# #BLOCK COMMENT SHORTCUT: cmd + / 
# #print(BIN_pts_df)
# print(len(grid_vectors.index))
# print(len(ray_vec_df.index))
# print(len(vectors_df.index))
# print(grid_vectors[0:10])
# print(grid_vectors[300:310])
# print(ray_vec_df[0:10])

print(grid_vectors.shape)
print(grid_vectors['0_x'].idxmax())
print(grid_vectors['0_x'].idxmin())
print(grid_vectors['1_x'].idxmax())
print(grid_vectors['1_x'].idxmin())
print(grid_vectors[609:615])

In [ ]:
# Plot test the grid vectors 

#Take a small portion of the vectors 
grid_slice = grid_vectors.truncate(after=600)

fig=plt.figure(figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111, projection='3d')
ax.quiver(grid_slice["0_x"], grid_slice["1_x"], grid_slice["2_x"], grid_slice["0_y"], grid_slice["1_y"], grid_slice["2_y"])

#ax.set_zlim(-1.01, 1.01)

ax.view_init(azim=0, elev=90)
plt.show()

In [ ]:
# Export dataframe to CSV
grid_vectors.to_csv(BINdir + "\\" + BIN + "\\" + BIN + "_rays.csv", \
                    sep=" ", header=False, index=False)

#### TAKES A WHILE TO SAVE! ####

In [ ]:
# Test that CSV saved properly 
csvFile = BINdir + "\\" + BIN + "\\" + BIN + "_rays.csv"

with open(csvFile, 'r') as f: 
    reader = csv.reader(f, delimiter = " ")
    data = list(reader)
    data = np.array(data).astype(float)

print(data.shape)
print(data[:3])

# Write batch file for floor

# -- OLD CODE --

In [ ]:
# Set up batch file and copy other input files to directory 

partsPath = dirPath + ' - parts'
masterBatch = simDir + '\\' + 'views_batch_master.bat'
masterBatRev = dirPath + '\\' + 'views_batch.bat'

# Copy input files into diretory 
otherFiles = ['sky.rad', 'geom.rad', 'geom.oct']
for i in otherFiles: 
    shutil.copyfile(partsPath + '\\' + i, dirPath + '\\' + i)

# Create the batch file
with open(masterBatch, 'r') as f: 
    masterBatFile = f.readlines()
    
for i in range(len(masterBatFile)): 
    masterBatFile[i] = masterBatFile[i].replace('cd', 'cd ' + dirPath)
    masterBatFile[i] = masterBatFile[i].replace('set skyfile=', 'set skyfile=' + dirPath + '\\' + 'sky.rad')
    masterBatFile[i] = masterBatFile[i].replace('set geofile=', 'set geofile=' + dirPath + '\\' + 'geom.rad')
    masterBatFile[i] = masterBatFile[i].replace('set octreefile=', 'set octreefile=' + dirPath + '\\' + 'geom.oct')
    masterBatFile[i] = masterBatFile[i].replace('set rayFile=', 'set rayFile=' + dirPath + '\\' + name + '.txt')
    masterBatFile[i] = masterBatFile[i].replace('set resultsFile=', 'set resultsFile=' + dirPath + '\\' + name + '.dat')

with open(masterBatRev, 'w') as out: 
    for i in masterBatFile: 
        out.write(i)
        

# Run batch file 